In [ ]:
# import libraries
!pip install tensorflow
# tensorflow_io 0.28 is compatible with TensorFlow 2.11
!pip install tensorflow_io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import pacakges
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

# **Load the Yamnet model**

In [ ]:
# load yamnet model
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [ ]:
# classes which the Yamnet model can classify into
class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
class_names =list(pd.read_csv(class_map_path)['display_name'])

for name in class_names[:20]:
  print(name)
print('...')

Speech
Child speech, kid speaking
Conversation
Narration, monologue
Babbling
Speech synthesizer
Shout
Bellow
Whoop
Yell
Children shouting
Screaming
Whispering
Laughter
Baby laughter
Giggle
Snicker
Belly laugh
Chuckle, chortle
Crying, sobbing
...


# **Data preprocessing**

In [ ]:
# Utility functions for loading audio files and making sure the sample rate is correct.
@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
# a fold column for train-valid split. The training data will be split into an 8:2 ratio. 
# Only 250 wav files are utilized for training because the extract_embeddings function finds N frames for one file, which can cause the number of training data to blow.
import random
all_folds = [1] * 320 + [5] * 80
random.shuffle(all_folds)

In [ ]:
# load the path of control (non-patient) and dementia (patient) datasets
import glob
control_filenames=glob.glob("/content/drive/MyDrive/Colab Notebooks/dementia/English/Pitt/train/control/*.wav")
control_labels=(len(control_filenames[0:200]))*[0]
demantia_filenames=glob.glob("/content/drive/MyDrive/Colab Notebooks/dementia/English/Pitt/train/dementia/*.wav")
dementia_labels=(len(demantia_filenames[0:200]))*[1]

In [ ]:
# combine two datasets into one 
all_filenames=control_filenames[0:200]+demantia_filenames[200:400]
all_labels=control_labels+dementia_labels

In [ ]:
# put training data into a DataFrame to visualize the training data
df = pd.DataFrame({'filename': all_filenames, 'label': all_labels, 'fold':all_folds})
print(df)

                                              filename  label  fold
0    /content/drive/MyDrive/Colab Notebooks/dementi...      0     5
1    /content/drive/MyDrive/Colab Notebooks/dementi...      0     1
2    /content/drive/MyDrive/Colab Notebooks/dementi...      0     1
3    /content/drive/MyDrive/Colab Notebooks/dementi...      0     1
4    /content/drive/MyDrive/Colab Notebooks/dementi...      0     1
..                                                 ...    ...   ...
395  /content/drive/MyDrive/Colab Notebooks/dementi...      1     1
396  /content/drive/MyDrive/Colab Notebooks/dementi...      1     1
397  /content/drive/MyDrive/Colab Notebooks/dementi...      1     1
398  /content/drive/MyDrive/Colab Notebooks/dementi...      1     1
399  /content/drive/MyDrive/Colab Notebooks/dementi...      1     1

[400 rows x 3 columns]


In [ ]:
# shuffle the Dataframe and the order of datasets during the training will be random
df=df.sample(frac=1)

In [ ]:
# load paths, labels and folds into a tf.Dataset objectt
filenames = df['filename']
labels = df['label']
folds = df['fold']

main_ds= tf.data.Dataset.from_tensor_slices((filenames, labels, folds))
main_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
def load_wav_for_map(filename, label, fold):
  return load_wav_16k_mono(filename), label, fold

In [ ]:
# load and resample the training data
main_ds = main_ds.map(load_wav_for_map)
main_ds.element_spec

(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
# Yamnet is a feature extractor to extract features from the audio file.
def extract_embedding(wav_data, label, fold):
  ''' run YAMNet to extract embedding from the wav data '''
  scores, embeddings, spectrogram = yamnet_model(wav_data)
  num_embeddings = tf.shape(embeddings[0:20])[0]
  return (embeddings[0:20],
            tf.repeat(label, num_embeddings),
            tf.repeat(fold, num_embeddings))

In [ ]:
# Extract embeddings using Yamnet
main_ds = main_ds.map(extract_embedding).unbatch()
main_ds.element_spec

(TensorSpec(shape=(1024,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
# split the data into train and validation set
cached_ds = main_ds.cache()
train_ds = cached_ds.filter(lambda embedding, label, fold: fold < 4)
val_ds = cached_ds.filter(lambda embedding, label, fold: fold == 5)

# remove the folds column now that it's not needed anymore
remove_fold_column = lambda embedding, label, fold: (embedding, label)

train_ds = train_ds.map(remove_fold_column)
val_ds = val_ds.map(remove_fold_column)

train_ds = train_ds.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

## Create the model


In [ ]:
# Build and compile the model
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
], name='my_model')

my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               524800    
                                                                 
 dense_15 (Dense)            (None, 2)                 1026      
                                                                 
Total params: 525,826
Trainable params: 525,826
Non-trainable params: 0
_________________________________________________________________


In [ ]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer='adam',
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=10,
                                            restore_best_weights=True)

In [ ]:
history = my_model.fit(train_ds,
                       epochs=50, batch_size=16, 
                       validation_data=val_ds,
                       callbacks=callback)

Epoch 1/50
2713/2713 [==============================] - 26s 9ms/step - loss: 0.4763 - accuracy: 0.7834 - val_loss: 0.7284 - val_accuracy: 0.6859
Epoch 2/50
2713/2713 [==============================] - 28s 10ms/step - loss: 0.4543 - accuracy: 0.7913 - val_loss: 0.7712 - val_accuracy: 0.6922
Epoch 3/50
2713/2713 [==============================] - 26s 9ms/step - loss: 0.4441 - accuracy: 0.7975 - val_loss: 0.6959 - val_accuracy: 0.6929
Epoch 4/50
2713/2713 [==============================] - 25s 9ms/step - loss: 0.4352 - accuracy: 0.8040 - val_loss: 0.8326 - val_accuracy: 0.6679
Epoch 5/50
2713/2713 [==============================] - 37s 14ms/step - loss: 0.4279 - accuracy: 0.8077 - val_loss: 0.9748 - val_accuracy: 0.6367
Epoch 6/50
2713/2713 [==============================] - 24s 9ms/step - loss: 0.4204 - accuracy: 0.8103 - val_loss: 0.8169 - val_accuracy: 0.6638
Epoch 7/50
2713/2713 [==============================] - 24s 9ms/step - loss: 0.4086 - accuracy: 0.8167 - val_loss: 0.8170 - val_

# Test the model using a ADRess 2021 data

In [ ]:
# Perform same data preprocessing as the training data
import glob
control_test_filenames=glob.glob('/content/drive/MyDrive/Colab Notebooks/dementia/English/ADReSS-2021/audio/control/*.wav')
control_test_labels=(len(control_test_filenames))*[0]
dementia_test_filenames=glob.glob('/content/drive/MyDrive/Colab Notebooks/dementia/English/ADReSS-2021/audio/dementia/*.wav')
dementia_test_labels=(len(dementia_test_filenames))*[1]
all_test_filenames=control_test_filenames+dementia_test_filenames
all_test_labels=control_test_labels+dementia_test_labels

In [ ]:
all_test_folds = [6]*1444

In [ ]:
test_ds=tf.data.Dataset.from_tensor_slices((all_test_filenames, all_test_labels, all_test_folds))

In [ ]:
test_ds = test_ds.map(load_wav_for_map)
test_ds.element_spec

(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None))

In [ ]:
test_ds = test_ds.map(extract_embedding).unbatch()
test_ds.element_spec

(TensorSpec(shape=(1024,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None))

In [ ]:
remove_fold_column = lambda embedding, label, fold: (embedding, label)

In [ ]:
test_ds = test_ds.map(remove_fold_column)

In [ ]:
test_ds = test_ds.cache().batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Evaluate the model using the test set
loss, accuracy = my_model.evaluate(test_ds, verbose=1)
print(loss)
print(accuracy)

100/100 [==============================] - 59s 542ms/step - loss: 1.1387 - Accuracy: 0.5000
1.1386741399765015
0.5


In [ ]:
# Save the model
my_model.save('/content/drive/MyDrive/Colab Notebooks/dementia/tensorflow_model_0510.h5')

In [ ]:
# Load the model
my_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/dementia/tensorflow_model.h5')